In [1]:
# importing networkx
import networkx as nx

# function to create a directed graph as specified by the input file. Edges are annotated with 'capacity' and 'weight' attributes, and nodes are annotated with 'demand' attributes
def create_graph(infile):
    myGraph=nx.DiGraph()
    myFile=open(infile,'r')
    
    while True:
        line = myFile.readline().strip()
        
        if line == "":
            break
        
        if line[0] == 'p':                  
            n = int(line.split()[2])
            myGraph.add_nodes_from(range(1, n+1), demand = 0)
            exist = [[0 for j in range(n)] for i in range(n)]
        
        elif line[0] == 'n':
            myGraph.node[int(line.split()[1])]['demand'] = int(line.split()[2])
        
        elif line[0] == 'a':
            line = [int(x) for x in line.split()[1:]]
            if exist[line[0]-1][line[1]-1] == 0: 
                myGraph.add_edge(line[0], line[1], capacity = line[3], weight = line[4])
                exist[line[0]-1][line[1]-1] = 1
            else:
                n = n+1
                myGraph.add_node(n, demand = 0)
                myGraph.add_edge(line[0], n, capacity = line[3], weight = line[4])
                myGraph.add_edge(n, line[1], capacity = line[3], weight = 0)
    
    return myGraph
    pass

In [2]:
# Checking the correctness
G_40 = create_graph('gte_bad.40')
G_6830 = create_graph('gte_bad.6830')
G_176280 = create_graph('gte_bad.176280')

print ("Correct value for _40 instance:", nx.min_cost_flow_cost(G_40) == 52099553858)
print ("Correct value for _6830 instance:", nx.min_cost_flow_cost(G_6830) == 299390431788)
print ("Correct value for _176280 instance:", nx.min_cost_flow_cost(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True


In [3]:
import pulp

# function to compute the value of the minimum cost flow by formulating and solving the problem as an LP
def lp_flow_value(G):
    problem=pulp.LpProblem("mincostflow", pulp.LpMinimize)
    capacity = nx.get_edge_attributes(G, 'capacity')
    weight = nx.get_edge_attributes(G, 'weight')
    demand = nx.get_node_attributes(G, 'demand')
    
    flow = pulp.LpVariable.dicts('flow', G.edges(), 0, None, pulp.LpInteger)
    problem += sum([weight[edge]*flow[edge] for edge in G.edges()]), "Total Flow Cost"
    
    for node in G.nodes():
        problem += sum(flow[in_edge] for in_edge in G.in_edges(node)) - sum(flow[out_edge] for out_edge in G.out_edges(node)) == demand[node]
    
    for edge in G.edges():
        problem += flow[edge] <= capacity[edge]
    
    problem.solve()
    
    return int(pulp.value(problem.objective))
    pass

In [4]:
print ("Correct value for _40 instance:", lp_flow_value(G_40) == 52099553858)
print ("Correct value for _6830 instance:", lp_flow_value(G_6830) == 299390431788)
print ("Correct value for _176280 instance:", lp_flow_value(G_176280) == 510585093810)

Correct value for _40 instance: True
Correct value for _6830 instance: True
Correct value for _176280 instance: True
